In [1]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statements_from_dataframe,extract_data_into_df,create_etl_watermark_table,insert_data_in_batches
from data_handler import execute_sql_script_from_config
from lookups import FileType
import prehook
from prehook import execute
import etl_handler
import posthook
db_session = create_connection()

In [2]:
posthook.execute()

V18_posthook.sql
SQL script executed successfully.


c:\Data_project\data_handler.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, engine)


In [3]:
etl_watermark = etl_handler.return_etl_watermark(db_session)
print(etl_watermark)


[Timestamp('2023-11-26 19:22:41.435659')]


In [4]:
etl_handler.update_or_insert_etl_watermark(db_session)

In [5]:
prehook.execute()

V0_prehook_Create_Schema.sql
SQL script executed successfully.
V1_prehook_stg_kaggle_spotify_tracks_table.sql
SQL script executed successfully.
V2_prehook_stg_kaggle_spotify_users_table.sql
SQL script executed successfully.
V3_prehook_genre_mapping_table.sql
SQL script executed successfully.
V4_prehook_Create_ETL_watermark_table.sql
SQL script executed successfully.
